<a href="https://colab.research.google.com/github/CyberIngeniero/Proyecto_Clustering_HDI/blob/master/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nuevos Clusters de Vehículos

# Librerias

In [ ]:
# Librerias
import os
import shutil

from google.colab import files    #files.upload()
from google.colab import drive    #drive.mount('/gdrive')

import pandas as pd
import numpy as np


# Carga y Lectura de Registros

In [ ]:
# Se crea directorio de trabajo
path = "/tmp/data_clusters"
os.mkdir(path)

In [ ]:
# Montando Drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# Copia del archivo a directorio local
!cp  /gdrive/My\ Drive/Proyecto_Nuevos_Clusters/Base_Clusters.csv /tmp/data_clusters

In [ ]:
# Moviendo datos a entorno de trabajo
os.rename("/tmp/data_clusters/Base_Clusters.csv", "/tmp/data_clusters/Base_Clusters_MM.csv")

In [ ]:
# Cambiando a directorio de trabajo
os.chdir(path)
print(os.getcwd())

/tmp/data_clusters


In [ ]:
df = pd.read_csv("Base_Clusters_MM.csv")
df.columns = df.columns.str.lower()
df = df.rename(columns = {'sum([cantidad convertida final])':'cantidad_polizas'})

In [ ]:
df.head()

,antiguedad_veh,prima_neta_deducible_anualizada,sisgen_usar,mod_tasacion_fiscal,precot_veh_modelo_id,canal_producto,precot_veh_tipo_id,precot_veh_tipo,precot_veh_uso,precot_veh_uso_id,numero_semana,edad,cotizacion,cantidad_polizas
0,0,"0,2","0,85",10,260013,DEALERS,1,AUTOMOVIL,Particular (Vehículos de uso personal y/o fami...,1,4,65,1,0
1,0,"0,24","0,85",32,2951001,DEALERS,1,AUTOMOVIL,Particular (Vehículos de uso personal y/o fami...,1,2,53,1,0
2,0,"0,27","0,85",39,2951006,DEALERS,1,AUTOMOVIL,Particular (Vehículos de uso personal y/o fami...,1,4,26,1,0
3,0,"0,31","1,1",-,2951054,DEALERS,1,AUTOMOVIL,Particular (Vehículos de uso personal y/o fami...,1,4,39,1,0
4,0,"0,99","0,85",-,521022,DIRECTO,6,STATION WAGON,Particular (Vehículos de uso personal y/o fami...,1,3,27,1,0


# Data Treatment

In [ ]:
df['precot_veh_uso'].unique()

array(['Particular (Vehículos de uso personal y/o familiar destinado al transporte sin fines de lucro)',
       'Vehículos de Reparto',
       'Otros usos comerciales distintos de transporte de pasajero y/o carga',
       'Transporte Pasajeros', 'Taxi Básico', 'Comercial'], dtype=object)